In [1]:
import os
from dotenv import load_dotenv

load_dotenv()

True

In [2]:
from langchain_core.tools import tool
import requests
import os
import math

def haversine_distance(lat1, lon1, lat2, lon2):
    """Calculate great-circle distance between two points on Earth."""
    R = 6371  # Earth radius in kilometers
    phi1, phi2 = math.radians(lat1), math.radians(lat2)
    d_phi = math.radians(lat2 - lat1)
    d_lambda = math.radians(lon2 - lon1)
    a = math.sin(d_phi / 2) ** 2 + math.cos(phi1) * math.cos(phi2) * math.sin(d_lambda / 2) ** 2
    c = 2 * math.atan2(math.sqrt(a), math.sqrt(1 - a))
    return R * c

@tool
def get_hotels_by_area_and_radius(
    bbox: str,
    arrival_date: str,
    departure_date: str,
    star_rating: str = "3,4,5",
    room_qty: int = 1,
    guest_qty: int = 1,
    children_qty: int = 0,
    children_age: str = "",
    currency: str = "USD",
    order_by: str = "popularity",
    categories_filter: str = "class::1,class::2,class::3",
    language: str = "en-us",
    travel_purpose: str = "leisure",
    offset: int = 0
) -> list:
    """
    Fetch hotel listings within a bounding box and return only key fields, sorted by distance to bbox center.

    Parameters:
        - bbox (str): Bounding box in format "min_lat,max_lat,min_lng,max_lng"
        - star_rating (str): Comma-separated star classes to filter, e.g., "3,4,5"
        - arrival_date (str): Check-in date (YYYY-MM-DD)
        - departure_date (str): Check-out date (YYYY-MM-DD)
        - room_qty (int): Number of rooms
        - guest_qty (int): Number of adults
        - children_qty (int): Number of children
        - children_age (str): Comma-separated list of children ages
        - currency (str): Price currency (e.g., USD, INR)
        - order_by (str): API sort preference (not used post-filter)
        - categories_filter (str): Used internally, overridden by star_rating
        - language (str): Response language
        - travel_purpose (str): "leisure" or "business"
        - offset (int): Pagination offset

    Returns:
        - list of hotel dicts sorted by ascending distance from bbox center.
    """

    # Convert star_rating to API-compatible format
    categories_filter = ",".join([f"class::{s.strip()}" for s in star_rating.split(",")])

    # Compute bbox center
    try:
        min_lat, max_lat, min_lng, max_lng = map(float, bbox.split(","))
        center_lat = (min_lat + max_lat) / 2
        center_lng = (min_lng + max_lng) / 2
    except Exception as e:
        return [{"error": f"Invalid bbox format: {e}"}]

    url = "https://apidojo-booking-v1.p.rapidapi.com/properties/list-by-map"
    querystring = {
        "room_qty": str(room_qty),
        "guest_qty": str(guest_qty),
        "bbox": bbox,
        "search_id": "none",
        "children_age": children_age,
        "price_filter_currencycode": currency,
        "categories_filter": categories_filter,
        "languagecode": language,
        "travel_purpose": travel_purpose,
        "children_qty": str(children_qty),
        "order_by": order_by,
        "offset": str(offset),
        "arrival_date": arrival_date,
        "departure_date": departure_date
    }
    headers = {
        "x-rapidapi-key": os.getenv("RAPIDAPI_KEY_HOTELS"),
        "x-rapidapi-host": "apidojo-booking-v1.p.rapidapi.com"
    }

    response = requests.get(url, headers=headers, params=querystring)
    if response.status_code != 200:
        return [{"error": response.text}]

    results = response.json().get("result", [])
    hotels = []

    for item in results:
        if not item.get("class"):
            continue

        lat = item.get("latitude")
        lng = item.get("longitude")
        if lat is None or lng is None:
            continue

        distance_km = haversine_distance(center_lat, center_lng, lat, lng)

        hotels.append({
            "name": item.get("hotel_name"),
            "star_rating": item.get("class"),
            "review_score": item.get("review_score"),
            "review_word": item.get("review_score_word"),
            "review_count": item.get("review_nr"),
            "address": item.get("address"),
            "city": item.get("city"),
            "district": item.get("district"),
            "latitude": lat,
            "longitude": lng,
            "price_per_night": item.get("min_total_price") or (
                item.get("price_breakdown", {}).get("all_inclusive_price")
            ),
            "image": item.get("main_photo_url"),
            "booking_url": item.get("url"),
            "is_free_cancellable": item.get("is_free_cancellable"),
            "is_mobile_deal": item.get("is_mobile_deal"),
            "checkin_from": item.get("checkin", {}).get("from"),
            "checkout_until": item.get("checkout", {}).get("until"),
            "distance_km": round(distance_km, 2)
        })

    # Sort by distance from bbox center
    hotels.sort(key=lambda h: h["distance_km"])
    return hotels

In [3]:
get_hotels_by_area_and_radius.description

'Fetch hotel listings within a bounding box and return only key fields, sorted by distance to bbox center.\n\nParameters:\n    - bbox (str): Bounding box in format "min_lat,max_lat,min_lng,max_lng"\n    - star_rating (str): Comma-separated star classes to filter, e.g., "3,4,5"\n    - arrival_date (str): Check-in date (YYYY-MM-DD)\n    - departure_date (str): Check-out date (YYYY-MM-DD)\n    - room_qty (int): Number of rooms\n    - guest_qty (int): Number of adults\n    - children_qty (int): Number of children\n    - children_age (str): Comma-separated list of children ages\n    - currency (str): Price currency (e.g., USD, INR)\n    - order_by (str): API sort preference (not used post-filter)\n    - categories_filter (str): Used internally, overridden by star_rating\n    - language (str): Response language\n    - travel_purpose (str): "leisure" or "business"\n    - offset (int): Pagination offset\n\nReturns:\n    - list of hotel dicts sorted by ascending distance from bbox center.'

In [4]:
get_hotels_by_area_and_radius.args

{'bbox': {'title': 'Bbox', 'type': 'string'},
 'arrival_date': {'title': 'Arrival Date', 'type': 'string'},
 'departure_date': {'title': 'Departure Date', 'type': 'string'},
 'star_rating': {'default': '3,4,5', 'title': 'Star Rating', 'type': 'string'},
 'room_qty': {'default': 1, 'title': 'Room Qty', 'type': 'integer'},
 'guest_qty': {'default': 1, 'title': 'Guest Qty', 'type': 'integer'},
 'children_qty': {'default': 0, 'title': 'Children Qty', 'type': 'integer'},
 'children_age': {'default': '', 'title': 'Children Age', 'type': 'string'},
 'currency': {'default': 'USD', 'title': 'Currency', 'type': 'string'},
 'order_by': {'default': 'popularity', 'title': 'Order By', 'type': 'string'},
 'categories_filter': {'default': 'class::1,class::2,class::3',
  'title': 'Categories Filter',
  'type': 'string'},
 'language': {'default': 'en-us', 'title': 'Language', 'type': 'string'},
 'travel_purpose': {'default': 'leisure',
  'title': 'Travel Purpose',
  'type': 'string'},
 'offset': {'defau

In [ ]:
response = get_hotels_by_area_and_radius.invoke({
    "bbox": "12.8300,12.9500,77.5000,77.6500", ## South Bangalore
    "arrival_date": "2025-07-10",
    "departure_date": "2025-07-13",
    "star_rating": "4,5"
})
response

[{'name': 'Greenpark Bengaluru',
  'star_rating': 5.0,
  'review_score': 8.5,
  'review_word': 'Very Good',
  'review_count': 384,
  'address': 'No. 75 & 172, Bannerghatta main road, J P Nagar',
  'city': 'Bengaluru',
  'district': '',
  'latitude': 12.89751,
  'longitude': 77.600046,
  'price_per_night': 17550,
  'image': 'https://cf.bstatic.com/xdata/images/hotel/square60/464861089.jpg?k=4a92dc79477e66a2eff4742b136f3a8ae44b557c6a02a6d4dfe0f4bb24f8b086&o=',
  'booking_url': 'https://www.booking.com/hotel/in/greenpark-bengaluru.html',
  'is_free_cancellable': 1,
  'is_mobile_deal': 0,
  'checkin_from': '14:00',
  'checkout_until': '12:00',
  'distance_km': 2.84},
 {'name': 'Super Collection O Bannergatta Road Near Decathlon',
  'star_rating': 4.0,
  'review_score': 7.8,
  'review_word': 'Good',
  'review_count': 60,
  'address': 'Himagiri, Gottigere, Near Decathlon, Bannerghatta Road',
  'city': 'Bangalore',
  'district': '',
  'latitude': 12.8614167,
  'longitude': 77.58975,
  'price_

In [6]:
response[0]

{'name': 'Greenpark Bengaluru',
 'star_rating': 5.0,
 'review_score': 8.5,
 'review_word': 'Very Good',
 'review_count': 384,
 'address': 'No. 75 & 172, Bannerghatta main road, J P Nagar',
 'city': 'Bengaluru',
 'district': '',
 'latitude': 12.89751,
 'longitude': 77.600046,
 'price_per_night': 17550,
 'image': 'https://cf.bstatic.com/xdata/images/hotel/square60/464861089.jpg?k=4a92dc79477e66a2eff4742b136f3a8ae44b557c6a02a6d4dfe0f4bb24f8b086&o=',
 'booking_url': 'https://www.booking.com/hotel/in/greenpark-bengaluru.html',
 'is_free_cancellable': 1,
 'is_mobile_deal': 0,
 'checkin_from': '14:00',
 'checkout_until': '12:00',
 'distance_km': 2.84}

In [7]:
response[1]

{'name': 'Super Collection O Bannergatta Road Near Decathlon',
 'star_rating': 4.0,
 'review_score': 7.8,
 'review_word': 'Good',
 'review_count': 60,
 'address': 'Himagiri, Gottigere, Near Decathlon, Bannerghatta Road',
 'city': 'Bangalore',
 'district': '',
 'latitude': 12.8614167,
 'longitude': 77.58975,
 'price_per_night': 3467.48,
 'image': 'https://cf.bstatic.com/xdata/images/hotel/square60/663567041.jpg?k=ab89b7dea9be23e85272ab67695b87ae436646fee9c44720c2acd8bab4e41907&o=',
 'booking_url': 'https://www.booking.com/hotel/in/collection-o-82501-blue-waters-lounge-bangalore.html',
 'is_free_cancellable': 1,
 'is_mobile_deal': 0,
 'checkin_from': '12:00',
 'checkout_until': '11:00',
 'distance_km': 3.56}

In [8]:
len(response)

13

In [9]:
response[2]

{'name': 'Super Townhouse BTM Layout Near Madiwala Lake Formerly Q Rooms',
 'star_rating': 4.0,
 'review_score': 7.6,
 'review_word': 'Good',
 'review_count': 190,
 'address': 'Plot Number 447, 7th Cross Road, Stage 2 BTM Layout, Bangalore',
 'city': 'Bangalore',
 'district': 'BTM Layout',
 'latitude': 12.9135034764044,
 'longitude': 77.6054028168904,
 'price_per_night': 5366.27,
 'image': 'https://cf.bstatic.com/xdata/images/hotel/square60/658120828.jpg?k=a73a9c609982f4d01fc2076f334ccf4832735bfd86087a3288d8783e07806e6e&o=',
 'booking_url': 'https://www.booking.com/hotel/in/oyo-townhouse-264-stage-2-btm-layout.html',
 'is_free_cancellable': 1,
 'is_mobile_deal': 0,
 'checkin_from': '12:00',
 'checkout_until': '11:00',
 'distance_km': 4.21}

In [30]:
response1 = get_hotels_by_area_and_radius.invoke({
    "bbox": "28.0185,28.1085,-82.4627,-82.3647", ## USF, Tampa, Florida 
    "arrival_date": "2025-07-10",
    "departure_date": "2025-07-13",
    "star_rating": "3,4,5"
})

response1

[{'name': 'Hyatt Place Tampa Busch Gardens',
  'star_rating': 3.0,
  'review_score': 8.0,
  'review_word': 'Very Good',
  'review_count': 795,
  'address': '11408 North 30th Street',
  'city': 'Tampa (Florida)',
  'district': '',
  'latitude': 28.052475283906,
  'longitude': -82.4280979313522,
  'price_per_night': 494,
  'image': 'https://cf.bstatic.com/xdata/images/hotel/square60/535388901.jpg?k=90b33f4d591bd52e55155a16605f9572b1ae3f206351fe0808f6015d6288bcd9&o=',
  'booking_url': 'https://www.booking.com/hotel/us/hyatt-place-tampa-busch-gardens.html',
  'is_free_cancellable': 1,
  'is_mobile_deal': 0,
  'checkin_from': '15:00',
  'checkout_until': '12:00',
  'distance_km': 1.87},
 {'name': 'OASIS BAY SUITES, Tampa, Busch Gardens, USF',
  'star_rating': 4.0,
  'review_score': 8.2,
  'review_word': 'Very Good',
  'review_count': 949,
  'address': '3001 University Center Drive',
  'city': 'Tampa',
  'district': 'Temple Terrace',
  'latitude': 28.0443636,
  'longitude': -82.4254149,
  'p

In [32]:
response1[0]

{'name': 'Hyatt Place Tampa Busch Gardens',
 'star_rating': 3.0,
 'review_score': 8.0,
 'review_word': 'Very Good',
 'review_count': 795,
 'address': '11408 North 30th Street',
 'city': 'Tampa (Florida)',
 'district': '',
 'latitude': 28.052475283906,
 'longitude': -82.4280979313522,
 'price_per_night': 494,
 'image': 'https://cf.bstatic.com/xdata/images/hotel/square60/535388901.jpg?k=90b33f4d591bd52e55155a16605f9572b1ae3f206351fe0808f6015d6288bcd9&o=',
 'booking_url': 'https://www.booking.com/hotel/us/hyatt-place-tampa-busch-gardens.html',
 'is_free_cancellable': 1,
 'is_mobile_deal': 0,
 'checkin_from': '15:00',
 'checkout_until': '12:00',
 'distance_km': 1.87}

In [33]:
response1[1]

{'name': 'OASIS BAY SUITES, Tampa, Busch Gardens, USF',
 'star_rating': 4.0,
 'review_score': 8.2,
 'review_word': 'Very Good',
 'review_count': 949,
 'address': '3001 University Center Drive',
 'city': 'Tampa',
 'district': 'Temple Terrace',
 'latitude': 28.0443636,
 'longitude': -82.4254149,
 'price_per_night': 375.3,
 'image': 'https://cf.bstatic.com/xdata/images/hotel/square60/588095891.jpg?k=b04f7558429a3498d7ed403077e0be284ea6f9f97c9085a04be456bf24134ea2&o=',
 'booking_url': 'https://www.booking.com/hotel/us/oasis-bay-suites.html',
 'is_free_cancellable': 1,
 'is_mobile_deal': 0,
 'checkin_from': '15:00',
 'checkout_until': '11:30',
 'distance_km': 2.42}

In [31]:
len(response1)

2

In [12]:
response1[0]

{'name': 'OASIS BAY SUITES, Tampa, Busch Gardens, USF',
 'star_rating': 4.0,
 'review_score': 8.2,
 'review_word': 'Very Good',
 'review_count': 949,
 'address': '3001 University Center Drive',
 'city': 'Tampa',
 'district': 'Temple Terrace',
 'latitude': 28.0443636,
 'longitude': -82.4254149,
 'price_per_night': 375.3,
 'image': 'https://cf.bstatic.com/xdata/images/hotel/square60/588095891.jpg?k=b04f7558429a3498d7ed403077e0be284ea6f9f97c9085a04be456bf24134ea2&o=',
 'booking_url': 'https://www.booking.com/hotel/us/oasis-bay-suites.html',
 'is_free_cancellable': 1,
 'is_mobile_deal': 0,
 'checkin_from': '15:00',
 'checkout_until': '11:30',
 'distance_km': 1.96}

In [13]:
response1[1]

{'name': 'Hyatt Place Tampa Busch Gardens',
 'star_rating': 3.0,
 'review_score': 8.0,
 'review_word': 'Very Good',
 'review_count': 795,
 'address': '11408 North 30th Street',
 'city': 'Tampa (Florida)',
 'district': '',
 'latitude': 28.052475283906,
 'longitude': -82.4280979313522,
 'price_per_night': 494,
 'image': 'https://cf.bstatic.com/xdata/images/hotel/square60/535388901.jpg?k=90b33f4d591bd52e55155a16605f9572b1ae3f206351fe0808f6015d6288bcd9&o=',
 'booking_url': 'https://www.booking.com/hotel/us/hyatt-place-tampa-busch-gardens.html',
 'is_free_cancellable': 1,
 'is_mobile_deal': 0,
 'checkin_from': '15:00',
 'checkout_until': '12:00',
 'distance_km': 1.99}

In [ ]:
response2 = get_hotels_by_area_and_radius.invoke({
    "bbox": "32.7550,32.8150,-96.8300,-96.7500",  ## texas, dallas
    "arrival_date": "2025-07-10",
    "departure_date": "2025-07-13",
    "star_rating": "3,4,5"
})

response2

[{'name': 'Mint House Dallas - Downtown',
  'star_rating': 3.0,
  'review_score': 9.3,
  'review_word': 'Wonderful',
  'review_count': 956,
  'address': '1601 Elm Street',
  'city': 'Dallas',
  'district': 'Main Street District',
  'latitude': 32.781927,
  'longitude': -96.798179,
  'price_per_night': 555.3,
  'image': 'https://cf.bstatic.com/xdata/images/hotel/square60/452429741.jpg?k=dfa580de9d8e3ee7b69df307237e0b272af2e71fd6b8376c5c4df74ff1317da2&o=',
  'booking_url': 'https://www.booking.com/hotel/us/the-guild-downtown-elm-st.html',
  'is_free_cancellable': 0,
  'is_mobile_deal': 1,
  'checkin_from': '15:00',
  'checkout_until': '11:00',
  'distance_km': 0.84},
 {'name': 'Hyatt Regency Dallas',
  'star_rating': 4.0,
  'review_score': 8.5,
  'review_word': 'Very Good',
  'review_count': 999,
  'address': '300 Reunion Boulevard',
  'city': 'Dallas (Texas)',
  'district': 'Downtown Dallas',
  'latitude': 32.7764796754412,
  'longitude': -96.8089860677719,
  'price_per_night': 627,
  '

In [15]:
len(response2)

5

In [16]:
response2[0]

{'name': 'Mint House Dallas - Downtown',
 'star_rating': 3.0,
 'review_score': 9.3,
 'review_word': 'Wonderful',
 'review_count': 956,
 'address': '1601 Elm Street',
 'city': 'Dallas',
 'district': 'Main Street District',
 'latitude': 32.781927,
 'longitude': -96.798179,
 'price_per_night': 555.3,
 'image': 'https://cf.bstatic.com/xdata/images/hotel/square60/452429741.jpg?k=dfa580de9d8e3ee7b69df307237e0b272af2e71fd6b8376c5c4df74ff1317da2&o=',
 'booking_url': 'https://www.booking.com/hotel/us/the-guild-downtown-elm-st.html',
 'is_free_cancellable': 0,
 'is_mobile_deal': 1,
 'checkin_from': '15:00',
 'checkout_until': '11:00',
 'distance_km': 0.84}

In [17]:
response2[1]

{'name': 'Hyatt Regency Dallas',
 'star_rating': 4.0,
 'review_score': 8.5,
 'review_word': 'Very Good',
 'review_count': 999,
 'address': '300 Reunion Boulevard',
 'city': 'Dallas (Texas)',
 'district': 'Downtown Dallas',
 'latitude': 32.7764796754412,
 'longitude': -96.8089860677719,
 'price_per_night': 627,
 'image': 'https://cf.bstatic.com/xdata/images/hotel/square60/199181724.jpg?k=4917e7203aecd96d01ea52c32ace43fe9ba79af9e2c50e188d7221b7c5950fb3&o=',
 'booking_url': 'https://www.booking.com/hotel/us/hyatt-regency-dallas.html',
 'is_free_cancellable': 1,
 'is_mobile_deal': 0,
 'checkin_from': '16:00',
 'checkout_until': '11:00',
 'distance_km': 2.01}

In [18]:
response2[2]

{'name': 'Locale Victory Park - Dallas',
 'star_rating': 5.0,
 'review_score': 9.2,
 'review_word': 'Wonderful',
 'review_count': 274,
 'address': '3111 North Houston St',
 'city': 'Dallas',
 'district': 'Uptown Dallas',
 'latitude': 32.7924044039174,
 'longitude': -96.8104961468812,
 'price_per_night': 957,
 'image': 'https://cf.bstatic.com/xdata/images/hotel/square60/436422388.jpg?k=1938e8d849c75e24ad39522d4cae0ed366a4fc0dee08821d4c3ad395be5278ca&o=',
 'booking_url': 'https://www.booking.com/hotel/us/the-guild-dallas-victory-park-katy-trail.html',
 'is_free_cancellable': 1,
 'is_mobile_deal': 0,
 'checkin_from': '15:00',
 'checkout_until': '11:00',
 'distance_km': 2.09}

In [19]:
response2[3]

{'name': 'Hyatt House Dallas Uptown',
 'star_rating': 3.0,
 'review_score': 7.7,
 'review_word': 'Good',
 'review_count': 1060,
 'address': '2914 Harry Hines Boulevard',
 'city': 'Dallas (Texas)',
 'district': 'Uptown Dallas',
 'latitude': 32.794589,
 'longitude': -96.809977,
 'price_per_night': 329.43,
 'image': 'https://cf.bstatic.com/xdata/images/hotel/square60/707312595.jpg?k=0e3132e27d3d3306ba6238d415261347418d5306fc49f226b209c5f2a8852385&o=',
 'booking_url': 'https://www.booking.com/hotel/us/hyatt-house-dallas-uptown.html',
 'is_free_cancellable': 1,
 'is_mobile_deal': 0,
 'checkin_from': '16:00',
 'checkout_until': '11:00',
 'distance_km': 2.15}

In [20]:
response3 = get_hotels_by_area_and_radius.invoke({
    "bbox": "22.5370,22.5670,88.3400,88.3700",  ## kolkata, park street
    "arrival_date": "2025-07-10",
    "departure_date": "2025-07-13",
    "star_rating": "3,4,5"
})

len(response3)

19

In [21]:
response3[0]

{'name': 'Park Suites',
 'star_rating': 3.0,
 'review_score': 8.1,
 'review_word': 'Very Good',
 'review_count': 632,
 'address': '31 Stephen Court, 18A Park Street',
 'city': 'Kolkata',
 'district': 'Park Street',
 'latitude': 22.552963,
 'longitude': 88.352319,
 'price_per_night': 14500,
 'image': 'https://cf.bstatic.com/xdata/images/hotel/square60/544302906.jpg?k=867705653d36741d7e83c81209691968987d98633b6bb942ef7e30db55139a0e&o=',
 'booking_url': 'https://www.booking.com/hotel/in/park-suites.html',
 'is_free_cancellable': 1,
 'is_mobile_deal': 0,
 'checkin_from': '14:00',
 'checkout_until': '12:00',
 'distance_km': 0.3}

In [22]:
response3[1]

{'name': 'FabHotel Heera Holiday Inn - Nr Eden Garden Stadium',
 'star_rating': 3.0,
 'review_score': 5.9,
 'review_word': 'Okay',
 'review_count': 38,
 'address': '51, 51,Elliot Road Park Street Near Assembly Of God Church Adjacent',
 'city': 'Kolkata',
 'district': '',
 'latitude': 22.55214657,
 'longitude': 88.35832329,
 'price_per_night': 5234.69,
 'image': 'https://cf.bstatic.com/xdata/images/hotel/square60/475280449.jpg?k=1d1f288b3bb82c563f173b42955fdbc6146378effb26539bd87bc8bf058fbb08&o=',
 'booking_url': 'https://www.booking.com/hotel/in/oyo-townhouse-201-heera-holiday-inn.html',
 'is_free_cancellable': 1,
 'is_mobile_deal': 1,
 'checkin_from': '12:00',
 'checkout_until': '11:00',
 'distance_km': 0.34}

In [23]:
response3[2]

{'name': "Super Collection O Aafreen Tower Near St. Thomas's Church",
 'star_rating': 4.0,
 'review_score': 4.0,
 'review_word': 'Disappointing',
 'review_count': 425,
 'address': '9/a, K Y D Street, Esplanade, Taltala, Kolkata, West Bengal',
 'city': 'Kolkata',
 'district': '',
 'latitude': 22.5557668157487,
 'longitude': 88.3540773577988,
 'price_per_night': 3713.6,
 'image': 'https://cf.bstatic.com/xdata/images/hotel/square60/698410557.jpg?k=ae2e0f7d13088faa2442a65e02d3febfcd30420d213ac5a934e2a2fc561dde40&o=',
 'booking_url': 'https://www.booking.com/hotel/in/collection-o-82694-aafreen-tower.html',
 'is_free_cancellable': 1,
 'is_mobile_deal': 0,
 'checkin_from': '12:00',
 'checkout_until': '11:00',
 'distance_km': 0.43}

In [24]:
response3[-1]

{'name': 'Polo Floatel Kolkata',
 'star_rating': 4.0,
 'review_score': 7.2,
 'review_word': 'Good',
 'review_count': 376,
 'address': '9/10 Kolkata Jetty, Opp Sbi Hq(Eastern Region), Strand Road, Kolkata',
 'city': 'Kolkata',
 'district': '',
 'latitude': 22.5664834583757,
 'longitude': 88.3401417732239,
 'price_per_night': 13632.3,
 'image': 'https://cf.bstatic.com/xdata/images/hotel/square60/564065544.jpg?k=9463474dbb77edfc9630b0fa920195e63516929696e82b500e67ad42602c5fcd&o=',
 'booking_url': 'https://www.booking.com/hotel/in/floatel-an-eco-friendly.html',
 'is_free_cancellable': 1,
 'is_mobile_deal': 0,
 'checkin_from': '14:00',
 'checkout_until': '12:00',
 'distance_km': 2.22}

In [2]:
from langchain_core.tools import tool
import os
import requests
from dotenv import load_dotenv

load_dotenv()

@tool
def get_places(city: str, query: str = "attractions") -> list:
    """
    Fetches top places (e.g., attractions, restaurants) in a city using the Foursquare Places API
    and returns them as a structured JSON list.

    Parameters:
        city (str): Name of the city (e.g., 'Kolkata')
        query (str): Type of places to search for (e.g., 'attractions', 'museums')

    Returns:
        list: A list of dictionaries, each containing:
            - name: Place name
            - categories: List of category names
            - address: Formatted address
            - phone: Telephone number if available
            - website: Website URL if available
"""
    api_key = os.getenv("FOURSQUARE_API_KEY")
    if not api_key:
        return [{"error": "Missing FOURSQUARE_API_KEY"}]

    url = "https://places-api.foursquare.com/places/search"
    headers = {
        "accept": "application/json",
        "X-Places-Api-Version": "2025-06-17",
        "authorization": api_key
    }
    params = {
        "near": city,
        "query": query,
        "limit": 10
    }

    response = requests.get(url, headers=headers, params=params)
    if response.status_code != 200:
        return [{"error": f"Foursquare API error: {response.text}"}]

    results = response.json().get("results", [])
    if not results:
        return [{"message": f"No results found for '{query}' in {city}."}]

    extracted = []
    for place in results:
        name = place.get("name", "Unknown")
        categories = [cat.get("name") for cat in place.get("categories", [])]
        address = place['location']['formatted_address']
        phone = place.get("tel", None)
        website = place.get("website", None)

        extracted.append({
            "name": name,
            "categories": categories,
            "address": address,
            "phone": phone,
            "website": website
        })

    return extracted

In [3]:
get_places.description

"Fetches top places (e.g., attractions, restaurants) in a city using the Foursquare Places API\nand returns them as a structured JSON list.\n\nParameters:\n    city (str): Name of the city (e.g., 'Kolkata')\n    query (str): Type of places to search for (e.g., 'attractions', 'museums')\n\nReturns:\n    list: A list of dictionaries, each containing:\n        - name: Place name\n        - categories: List of category names\n        - address: Formatted address\n        - phone: Telephone number if available\n        - website: Website URL if available"

In [4]:
get_places.args

{'city': {'title': 'City', 'type': 'string'},
 'query': {'default': 'attractions', 'title': 'Query', 'type': 'string'}}

In [5]:
get_places.invoke(
    {
        'city': 'kolkata',
        'query': 'attractions'
    }
)

[{'name': 'Victoria Memorial',
  'categories': ['History Museum'],
  'address': "1, Queen's Way, Kolkata 700071, West Bengal",
  'phone': '033 2223 1890',
  'website': 'http://www.victoriamemorial-cal.org'},
 {'name': 'Science City',
  'categories': ['Amusement Park', 'Science Museum'],
  'address': 'Dhapa, E.M. Bypass (J.B.S. Haldane Avenue), Kolkata 700046, West Bengal',
  'phone': '033 2285 2607',
  'website': 'http://sciencecitykolkata.org.in'},
 {'name': 'Interior Designing Trends',
  'categories': ['Furniture and Home Store'],
  'address': '34A, Sashi Bhushan Dey St, Kolkata 700012, West Bengal',
  'phone': None,
  'website': None},
 {'name': 'Body Massage Parlor',
  'categories': ['Massage Clinic'],
  'address': 'Acharya Jagadish Chandra Bose Rd, Kolkata 700017, West Bengal',
  'phone': '098743 82581',
  'website': 'http://www.greenviewmassageparlour.com'},
 {'name': 'Nicco Park',
  'categories': ['Amusement Park'],
  'address': 'Sector IV, Salt Lake City, Kolkata 700106, West B

In [6]:
from langchain_core.tools import tool
import os
import requests
from dotenv import load_dotenv
from datetime import datetime, timedelta

load_dotenv()

@tool
def get_hotels_by_city(city: str) -> list:
    """
    Get top hotels with prices per night in INR for the given city using Hotels4 (RapidAPI).
    
    Parameters:
        city (str): City name (e.g., "Kolkata")

    Returns:
        list: List of hotels with name, price, and address
    """

    headers = {
        "x-rapidapi-key": os.getenv("RAPIDAPI_KEY"),
        "x-rapidapi-host": "hotels4.p.rapidapi.com"
    }

    # Step 1: Get gaiaId (destination identifier)
    location_url = "https://hotels4.p.rapidapi.com/locations/v3/search"
    location_params = {
        "q": city,
        "locale": "en_US",
        "langid": "1033",
        "siteid": "3000000"
    }

    location_resp = requests.get(location_url, headers=headers, params=location_params)
    if location_resp.status_code != 200:
        return [{"error": f"Location fetch failed: {location_resp.text}"}]

    try:
        gaia_id = location_resp.json()["sr"][0]["gaiaId"]
    except Exception:
        return [{"error": f"Could not find destination ID for city: {city}"}]

    # Step 2: Get hotel list for destination
    hotel_url = "https://hotels4.p.rapidapi.com/properties/v2/list"
    today = datetime.today()
    checkin_date = today.strftime("%Y-%m-%d")
    checkout_date = (today + timedelta(days=1)).strftime("%Y-%m-%d")

    payload = {
        "currency": "INR",
        "locale": "en_US",
        "siteId": 3000000,
        "destination": {"regionId": gaia_id},
        "checkInDate": {
            "day": today.day,
            "month": today.month,
            "year": today.year
        },
        "checkOutDate": {
            "day": (today + timedelta(days=1)).day,
            "month": (today + timedelta(days=1)).month,
            "year": (today + timedelta(days=1)).year
        },
        "rooms": [{"adults": 1}],
        "resultsStartingIndex": 0,
        "resultsSize": 25,
        # "sort": "PRICE_LOW_TO_HIGH",
        "filters": {}
    }

    hotel_resp = requests.post(hotel_url, json=payload, headers=headers)
    if hotel_resp.status_code != 200:
        return [{"error": f"Hotel list fetch failed: {hotel_resp.text}"}]

    hotels_raw = hotel_resp.json().get("data", {}).get("propertySearch", {}).get("properties", [])
    if not hotels_raw:
        return [{"message": f"No hotel results found in {city}"}]

    results = []
    for hotel in hotels_raw:
        print(hotel)
        print('*'*50)
        print()
        name = hotel.get("name")
        address = hotel.get("address", {}).get("addressLine", "No address provided")
        price = hotel.get("price", {}).get("lead", {}).get("formatted", "N/A")
        results.append({
            "name": name,
            "address": address,
            "price_per_night": price
        })

    return results

In [7]:
get_hotels_by_city.invoke({"city": "Kolkata"})

[{'error': 'Location fetch failed: {"message":"Invalid API key. Go to https:\\/\\/docs.rapidapi.com\\/docs\\/keys for more info."}'}]

In [8]:
## currency exchange tool
from langchain_core.tools import tool
import requests

@tool
def convert_currency(amount: float, to_currency: str, base: str = "USD") -> float:
    """
    Convert `amount` from `base` currency to `to_currency` using ExchangeRate-API open endpoint.
    """
    url = f"https://open.er-api.com/v6/latest/{base}"
    resp = requests.get(url)
    data = resp.json()
    rate = data["rates"].get(to_currency)
    if not rate:
        return {"error": f"Rate unavailable for {to_currency}"}
    return round(amount * rate, 2)

In [9]:
convert_currency.args

{'amount': {'title': 'Amount', 'type': 'number'},
 'to_currency': {'title': 'To Currency', 'type': 'string'},
 'base': {'default': 'USD', 'title': 'Base', 'type': 'string'}}

In [10]:
convert_currency.invoke(
    {
        'amount': 1,
        'to_currency': 'INR'
    }
)

85.54

In [11]:
from langchain_core.tools import tool
from dotenv import load_dotenv
import requests
import os

load_dotenv()

@tool
def get_flight_fares(from_code: str, to_code: str, date: str, adult: int = 1, type_: str = "economy") -> list:
    """
    Fetches flight fare data using the Flight Fare Search API on RapidAPI.

    Args:
        from_code (str): IATA code of departure airport (e.g., 'BLR')
        to_code (str): IATA code of arrival airport (e.g., 'CCU')
        date (str): Travel date in YYYY-MM-DD
        adult (int): Number of adult passengers (default: 1)
        type_ (str): Cabin class (default: 'economy')

    Returns:
        list: List of flights with key details: timing, pricing, stops, countries, and cabin info.
    """
    url = "https://flight-fare-search.p.rapidapi.com/v2/flights/"

    querystring = {
        "from": from_code,
        "to": to_code,
        "date": date,
        "adult": str(adult),
        "type": type_,
        "currency": "USD"
    }

    headers = {
        "x-rapidapi-key": os.getenv("RAPIDAPI_KEY"),
        "x-rapidapi-host": "flight-fare-search.p.rapidapi.com"
    }

    response = requests.get(url, headers=headers, params=querystring)
    print("🔍 Raw API response:", response.status_code, response.text)

    try:
        raw = response.json()
        flights = raw.get("results", [])
        if not isinstance(flights, list) or not flights:
            return [{"message": "No flights found."}]

        results = []
        for f in flights:
            stop_info = []
            stop_summary = f.get("stopSummary", {})

            # Extract intermediate stops if present
            if isinstance(stop_summary, dict):
                for key, val in stop_summary.items():
                    if key != "connectingTime" and isinstance(val, dict):
                        stop_info.append({
                            "intermediate_airport": val.get("airport", "Unknown"),
                            "stop_duration_minutes": val.get("stopDuration")
                        })

            results.append({
                "flight_code": f.get("flight_code"),
                "airline": f.get("flight_name"),
                "cabin_type": f.get("cabinType", "Unknown"),
                "stops": f.get("stops", "Unknown"),
                "departure_city": f.get("departureAirport", {}).get("city"),
                "departure_country": f.get("departureAirport", {}).get("country", {}).get("label"),
                "departure_time": f.get("departureAirport", {}).get("time"),
                "arrival_city": f.get("arrivalAirport", {}).get("city"),
                "arrival_country": f.get("arrivalAirport", {}).get("country", {}).get("label"),
                "arrival_time": f.get("arrivalAirport", {}).get("time"),
                "duration": f.get("duration", {}).get("text"),
                "price": f.get("totals", {}).get("total"),
                "currency": f.get("totals", {}).get("currency"),
                "intermediate_stops": stop_info if stop_info else None
            })

        return results

    except Exception as e:
        return [{"error": str(e)}]

In [12]:
get_flight_fares.description

"Fetches flight fare data using the Flight Fare Search API on RapidAPI.\n\nArgs:\n    from_code (str): IATA code of departure airport (e.g., 'BLR')\n    to_code (str): IATA code of arrival airport (e.g., 'CCU')\n    date (str): Travel date in YYYY-MM-DD\n    adult (int): Number of adult passengers (default: 1)\n    type_ (str): Cabin class (default: 'economy')\n\nReturns:\n    list: List of flights with key details: timing, pricing, stops, countries, and cabin info."

In [13]:
get_flight_fares.args

{'from_code': {'title': 'From Code', 'type': 'string'},
 'to_code': {'title': 'To Code', 'type': 'string'},
 'date': {'title': 'Date', 'type': 'string'},
 'adult': {'default': 1, 'title': 'Adult', 'type': 'integer'},
 'type_': {'default': 'economy', 'title': 'Type', 'type': 'string'}}

In [14]:
response = get_flight_fares.invoke({
    "from_code": "BLR",
    "to_code": "CCU",
    "date": "2025-07-01"
})

print(response)

🔍 Raw API response: 401 {"message":"Invalid API key. Go to https:\/\/docs.rapidapi.com\/docs\/keys for more info."}
[{'message': 'No flights found.'}]


In [15]:
response

[{'message': 'No flights found.'}]

{'id': '598773c820cf2ad796f1771aa043f6f019f262ac6b148bf6a725673a57a10086', 'careerCode': 'IX', 'flight_code': 'IX-5610', 'flight_name': 'Air India Express', 'stops': '1 Stop', 'cabinType': 'Economy', 'baggage': {'cabin': {'allowance': 7, 'qty': 1, 'unit': 'KG', 'text': '7 KG'}, 'checkIn': {'allowance': 15, 'qty': 1, 'unit': 'KG', 'text': '15 KG', 'totalWeight': 15, 'quantity': 1}, 'baggageOptionsAvailable': False}, 'currency': 'USD', 'departureAirport': {'time': '2025-07-01T05:00:00', 'code': 'BLR', 'tz': 'Asia/Kolkata', 'timeZone': '5.50', 'type': '2', 'label': 'Bengaluru International Airport', 'country': {'label': 'India', 'code': 'IN'}, 'city': 'Bangalore'}, 'arrivalAirport': {'time': '2025-07-01T12:20:00', 'code': 'CCU', 'tz': 'Asia/Kolkata', 'timeZone': '5.50', 'type': '2', 'label': 'Netaji Subhas Chandra Bose Airport', 'country': {'label': 'India', 'code': 'IN'}, 'city': 'Kolkata'}, 'path': ['IX-5610', 'IX-1256'], 'duration': {'text': '07h 20m', 'value': 440}, 'stopSummary': {'0': {'airport': 'MAA', 'stopDuration': 215}, 'connectingTime': None}, 'totals': {'currency': 'USD', 'baggage': None, 'penalty': None, 'total': 55.207840000000004, 'tax': None, 'base': 55.207840000000004}}


{'id': 'd8f07b3f1c1c0d13445db7a76a89a07ba0eade2b70ab4f513a2b528e71df846f', 'careerCode': 'IX', 'flight_code': 'IX-5012', 'flight_name': 'Air India Express', 'stops': '1 Stop', 'cabinType': 'Economy', 'baggage': {'cabin': {'allowance': 7, 'qty': 1, 'unit': 'KG', 'text': '7 KG'}, 'checkIn': {'allowance': 15, 'qty': 1, 'unit': 'KG', 'text': '15 KG', 'totalWeight': 15, 'quantity': 1}, 'baggageOptionsAvailable': False}, 'currency': 'USD', 'departureAirport': {'time': '2025-07-01T05:15:00', 'code': 'BLR', 'tz': 'Asia/Kolkata', 'timeZone': '5.50', 'type': '2', 'label': 'Bengaluru International Airport', 'country': {'label': 'India', 'code': 'IN'}, 'city': 'Bangalore'}, 'arrivalAirport': {'time': '2025-07-01T18:30:00', 'code': 'CCU', 'tz': 'Asia/Kolkata', 'timeZone': '5.50', 'type': '2', 'label': 'Netaji Subhas Chandra Bose Airport', 'country': {'label': 'India', 'code': 'IN'}, 'city': 'Kolkata'}, 'path': ['IX-5012', 'IX-1224'], 'duration': {'text': '13h 15m', 'value': 795}, 'stopSummary': {'0': {'airport': 'HYD', 'stopDuration': 330}, 'connectingTime': None}, 'totals': {'currency': 'USD', 'baggage': None, 'penalty': None, 'total': 61.833760000000005, 'tax': None, 'base': 61.833760000000005}}


{'id': '74afe26b11cfaf84ebbc4166f3d706e270f311194100e36d474fe2a3506cb3cc', 'careerCode': 'IX', 'flight_code': 'IX-5012', 'flight_name': 'Air India Express', 'stops': '1 Stop', 'cabinType': 'Economy', 'baggage': {'cabin': {'allowance': 7, 'qty': 1, 'unit': 'KG', 'text': '7 KG'}, 'checkIn': {'allowance': 15, 'qty': 1, 'unit': 'KG', 'text': '15 KG', 'totalWeight': 15, 'quantity': 1}, 'baggageOptionsAvailable': False}, 'currency': 'USD', 'departureAirport': {'time': '2025-07-01T05:15:00', 'code': 'BLR', 'tz': 'Asia/Kolkata', 'timeZone': '5.50', 'type': '2', 'label': 'Bengaluru International Airport', 'country': {'label': 'India', 'code': 'IN'}, 'city': 'Bangalore'}, 'arrivalAirport': {'time': '2025-07-01T20:25:00', 'code': 'CCU', 'tz': 'Asia/Kolkata', 'timeZone': '5.50', 'type': '2', 'label': 'Netaji Subhas Chandra Bose Airport', 'country': {'label': 'India', 'code': 'IN'}, 'city': 'Kolkata'}, 'path': ['IX-5012', 'IX-2825'], 'duration': {'text': '15h 10m', 'value': 910}, 'stopSummary': {'0': {'airport': 'HYD', 'stopDuration': 445}, 'connectingTime': None}, 'totals': {'currency': 'USD', 'baggage': None, 'penalty': None, 'total': 61.833760000000005, 'tax': None, 'base': 61.833760000000005}}


{'id': 'f1ba1853c96aba6b6367411e6152c0f663a902d8a8fcb1226a0263cd5f674236', 'careerCode': 'IX', 'flight_code': 'IX-2870', 'flight_name': 'Air India Express', 'stops': '1 Stop', 'cabinType': 'Economy', 'baggage': {'cabin': {'allowance': 7, 'qty': 1, 'unit': 'KG', 'text': '7 KG'}, 'checkIn': {'allowance': 15, 'qty': 1, 'unit': 'KG', 'text': '15 KG', 'totalWeight': 15, 'quantity': 1}, 'baggageOptionsAvailable': False}, 'currency': 'USD', 'departureAirport': {'time': '2025-07-01T07:35:00', 'code': 'BLR', 'tz': 'Asia/Kolkata', 'timeZone': '5.50', 'type': '2', 'label': 'Bengaluru International Airport', 'country': {'label': 'India', 'code': 'IN'}, 'city': 'Bangalore'}, 'arrivalAirport': {'time': '2025-07-01T18:30:00', 'code': 'CCU', 'tz': 'Asia/Kolkata', 'timeZone': '5.50', 'type': '2', 'label': 'Netaji Subhas Chandra Bose Airport', 'country': {'label': 'India', 'code': 'IN'}, 'city': 'Kolkata'}, 'path': ['IX-2870', 'IX-1224'], 'duration': {'text': '10h 55m', 'value': 655}, 'stopSummary': {'0': {'airport': 'HYD', 'stopDuration': 330}, 'connectingTime': None}, 'totals': {'currency': 'USD', 'baggage': None, 'penalty': None, 'total': 61.833760000000005, 'tax': None, 'base': 61.833760000000005}}


{'id': '3cc0a45d6ed4ef6c8a72807c3cc9e9c0b5c5964cb930780c9f8785868460f7f6', 'careerCode': 'IX', 'flight_code': 'IX-1574', 'flight_name': 'Air India Express', 'stops': 'Direct', 'cabinType': 'Economy', 'baggage': {'cabin': {'allowance': 7, 'qty': 1, 'unit': 'KG', 'text': '7 KG'}, 'checkIn': {'allowance': 15, 'qty': 1, 'unit': 'KG', 'text': '15 KG', 'totalWeight': 15, 'quantity': 1}, 'baggageOptionsAvailable': False}, 'currency': 'USD', 'departureAirport': {'time': '2025-07-01T14:30:00', 'code': 'BLR', 'tz': 'Asia/Kolkata', 'timeZone': '5.50', 'type': '2', 'label': 'Bengaluru International Airport', 'country': {'label': 'India', 'code': 'IN'}, 'city': 'Bangalore'}, 'arrivalAirport': {'time': '2025-07-01T17:20:00', 'code': 'CCU', 'tz': 'Asia/Kolkata', 'timeZone': '5.50', 'type': '2', 'label': 'Netaji Subhas Chandra Bose Airport', 'country': {'label': 'India', 'code': 'IN'}, 'city': 'Kolkata'}, 'path': ['IX-1574'], 'duration': {'text': '02h 50m', 'value': 170}, 'stopSummary': {'connectingTime': None}, 'totals': {'currency': 'USD', 'baggage': None, 'penalty': None, 'total': 74.02752000000001, 'tax': None, 'base': 74.02752000000001}}


In [16]:
response = get_flight_fares.invoke({
    "from_code": "DEL",
    "to_code": "BOM",
    "date": "2025-07-02"
})

response

🔍 Raw API response: 429 {"message":"Too many requests"}


[{'message': 'No flights found.'}]

In [17]:
query = {
    "from_code": "BLR",
    "to_code": "BKK",
    "date": "2025-07-02"
}

response = get_flight_fares.invoke(query)
response

🔍 Raw API response: 429 {"message":"Too many requests"}


[{'message': 'No flights found.'}]